In [1]:
#Import Modules
import os
import deconvolution_tools as dt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline


In [2]:
drive_path = '/Volumes/Brain2017/data/dynamic-brain-workshop/brain_observatory_cache/'

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
manifest_file = os.path.join(drive_path,'brain_observatory_manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [3]:
pd.DataFrame(boc.get_experiment_containers()).head()

,cre_line,donor_name,failed,id,imaging_depth,reporter_line,specimen_name,tags,targeted_structure
0,Cux2-CreERT2,283284,False,566759225,275,Ai93(TITL-GCaMP6f),Cux2-CreERT2;Camk2a-tTA;Ai93-283284,[],VISam
1,Nr5a1-Cre,261969,False,546328009,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-261969,[],VISal
2,Scnn1a-Tg3-Cre,230570,False,511510911,350,Ai93(TITL-GCaMP6f),Scnn1a-Tg3-Cre;Camk2a-tTA;Ai93-230570,[],VISp
3,Emx1-IRES-Cre,284669,False,569792815,375,Ai93(TITL-GCaMP6f),Emx1-IRES-Cre;Camk2a-tTA;Ai93-284669,[],VISam
4,Rbp4-Cre_KL100,234584,False,511511015,375,Ai93(TITL-GCaMP6f),Rbp4-Cre;Camk2a-tTA;Ai93-234584,[],VISpm


In [4]:
sessions = boc.get_ophys_experiments(experiment_container_ids=[569792815])
pd.DataFrame(sessions).head()

,acquisition_age_days,cre_line,donor_name,experiment_container_id,id,imaging_depth,reporter_line,session_type,specimen_name,targeted_structure
0,128,Emx1-IRES-Cre,284669,569792815,569792817,375,Ai93(TITL-GCaMP6f),three_session_A,Emx1-IRES-Cre;Camk2a-tTA;Ai93-284669,VISam
1,137,Emx1-IRES-Cre,284669,569792815,571006351,375,Ai93(TITL-GCaMP6f),three_session_C2,Emx1-IRES-Cre;Camk2a-tTA;Ai93-284669,VISam
2,136,Emx1-IRES-Cre,284669,569792815,570889097,375,Ai93(TITL-GCaMP6f),three_session_B,Emx1-IRES-Cre;Camk2a-tTA;Ai93-284669,VISam


In [5]:
ophys_experiment_id = 569792817	 #example experiment

dt.get_epoch_spiking_data(boc=boc, ophys_experiment_id=ophys_experiment_id,std=10)

print 'a'




/Users/perryspratt/anaconda/lib/python2.7/site-packages/OASIS/functions.py:15: UserWarning: Could not find cvxpy. Don't worry, you can still use OASIS, just not the slower interior point methods we compared to in the papers.
  "just not the slower interior point methods we compared to in the papers.")


a


In [10]:
dff_traces, timestamps, cell_specimen_ids = dt.get_dff(boc=boc, ophys_experiment_id=ophys_experiment_id)

from OASIS.functions import deconvolve

dff_trace = dff_traces[0]
num_std = 6
penalty = 1
spike_prob_list = []
dff = {}

for i, dff_trace in enumerate(dff_traces):
    c, s, b, g, lam = deconvolve(np.double(dff_trace), penalty=penalty)
    spike_prob_list.append(s)
    dff[cell_specimen_ids[i]] = dff_trace

In [7]:
def change_deconv_significance(spike_prob_list,cell_specimen_ids, num_std):
    std = np.std(np.asarray(spike_prob_list))

    spike_times = {}
    isis = {}
    spikes = {}

    for i, spike_prob in enumerate(spike_prob_list):
        s_sig = (spike_prob >= num_std * std)
        spikes[cell_specimen_ids[i]] = s_sig * 1
        spike_times[cell_specimen_ids[i]] = timestamps[s_sig]
        isis[cell_specimen_ids[i]] = np.diff(spike_times[cell_specimen_ids[i]])
    
    return spikes

    

In [8]:
def get_firing_rate(spikes, timestamps, window):
    fr = []
    time = timestamps[window*2] - timestamps[0]
    for i in range(window, len(spikes)-window):
        inst_fr = np.sum(spikes[i-window:i+window])/time
        fr.append(inst_fr)

    
    return fr
    

In [9]:
start = 2500
stop =  3000

for num_std in range(5):
    cell_ids = spikes.keys()
    index = 0
    
    spikes = change_deconv_significance(spike_prob_list,cell_specimen_ids, num_std)
    
    print np.sum(spikes[cell_ids[index]])
    
    plt.plot(dff[cell_ids[index]])
    plt.plot(spikes[cell_ids[index]] * np.max(dff[cell_ids[index]]))
    plt.xlim(start, stop)
    plt.title(str(num_std) + ' STD: ' + str(np.sum(spikes[cell_ids[index]])) + ' spikes')
    plt.show()
    
    fr = get_firing_rate(spikes[cell_ids[index]], timestamps, 15)

    plt.plot(fr)
    plt.title(str(num_std)+ ' STD: Mean FR=' + str(np.mean(fr)))
    plt.show()
    

NameError: name 'spikes' is not defined

In [ ]:
for num_std in range(5):
    spikes = change_deconv_significance(spike_prob_list,cell_specimen_ids, num_std)
    fr = []
    mean_fr = []
    for i,k in enumerate(spikes):
#         fr.append(get_firing_rate(spikes[k], timestamps, 15))
        mean_fr.append(np.mean(get_firing_rate(spikes[k], timestamps, 15)))
    plt.hist(mean_fr)
    plt.title(str(num_std)+ ' STD')
    plt.show()
    
    
    